# Spacegroup determination: comparing spglib vs neural network based on the diffraction intensity in spherical harmonics (DISH) descriptor

Author: Angelo Ziletti (angelo.ziletti@gmail.com; ziletti@fhi-berlin.mpg.de)

In [1]:
from ai4materials.dataprocessing.preprocessing import load_dataset_from_file
from ai4materials.wrappers import load_descriptor
from ai4materials.utils.utils_config import set_configs
from ai4materials.utils.utils_config import setup_logger
from ai4materials.dataprocessing.preprocessing import load_dataset_from_file
from ai4materials.dataprocessing.preprocessing import prepare_dataset
from ai4materials.utils.utils_crystals import random_displace_atoms
from ai4materials.utils.utils_crystals import get_spacegroup, get_spacegroup_analyzer
from ase.spacegroup import get_spacegroup as ase_get_spacegroup
from collections import Counter
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
%matplotlib inline  

main_folder = '/home/ziletti/Documents/calc_nomadml/rot_inv_3d/'
dataset_folder = os.path.abspath(os.path.normpath(os.path.join(main_folder, 'datasets')))
desc_folder = os.path.abspath(os.path.normpath(os.path.join(main_folder, 'desc_folder')))

configs = set_configs(main_folder=main_folder)
logger = setup_logger(configs, level='INFO', display_configs=False)
configs['io']['dataset_folder'] = dataset_folder
configs['io']['desc_folder'] = desc_folder

/home/ziletti/anaconda2/lib/python2.7/site-packages/pymatgen/__init__.py:87: UserWarning: 
Pymatgen will drop Py2k support from v2019.1.1. Pls consult the documentation
at https://www.pymatgen.org for more details.
  at https://www.pymatgen.org for more details.""")


## Loading prototype database and check the classification for pristine structures

In [2]:
db_files_prototypes_basedir = '/home/ziletti/Documents/calc_nomadml/rot_inv_3d/db_ase/'

proto_names = ['A_cP1_221_a']

prototypes_basedir = '/home/ziletti/Documents/calc_nomadml/rot_inv_3d/prototypes_aflow_new'
ase_db_files = [os.path.join(db_files_prototypes_basedir, proto_name) + '.db' for proto_name in proto_names]

db_protos = zip(proto_names, ase_db_files)

In [3]:
from ai4materials.utils.utils_data_retrieval import read_ase_db
from ai4materials.utils.utils_crystals import create_supercell
import random 

y_pred = []
for idx_db, db_proto in enumerate(db_protos):
    ase_atoms_list = read_ase_db(db_path=ase_db_files[idx_db])

In [4]:
print(len(ase_atoms_list[0]))

1


## Classification results spglib

In [5]:
y_pred = []
y_true = []
ase_atoms_list = ase_atoms_list[:5]

y_true = y_true + [221]*len(ase_atoms_list)

for idx, structure in enumerate(ase_atoms_list):
    y_pred.append(ase_get_spacegroup(structure, symprec=1e-3).no)

In [6]:
accuracy_score(y_true, y_pred)

1.0

In [7]:
Counter(y_true), Counter(y_pred)

(Counter({221: 5}), Counter({221: 5}))

In [8]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_true, y_pred)
np.set_printoptions(precision=2)
cnf_matrix

array([[5]])

221
221
221
221
221


## Supercell creation

In [56]:
y_pred = []
y_pred_sc1 = []
y_pred_sc2 = []
y_pred_sc3 = []

ase_atoms_list_supercell = []

for idx, structure in enumerate(ase_atoms_list):
    #supercell =  create_supercell(structure, create_replicas_by='nb_atoms', 
    #                              target_nb_atoms=128, optimal_supercell=True)
    
    #supercell =  random_displace_atoms(structure, displacement_scaled=1e-12, 
    #                                   create_replicas_by='nb_atoms', 
    #                                   noise_distribution='uniform_scaled', 
    #                                   target_nb_atoms=128,
    #                                   optimal_supercell=True)

    supercell =  random_displace_atoms(structure, displacement_scaled=0.006, 
                                       create_replicas_by='nb_atoms', 
                                       noise_distribution='uniform_scaled', 
                                       target_nb_atoms=128,
                                       optimal_supercell=True)
        
    alpha = random.random() * 360.0
    structure.rotate(alpha, 'x', rotate_cell=True, center='COU')

    beta = random.random() * 360.0
    structure.rotate(beta, 'y', rotate_cell=True, center='COU')

    gamma = random.random() * 360.0
    structure.rotate(gamma, 'z', rotate_cell=True, center='COU')
    
    ase_atoms_list_supercell.append(supercell)
    
    get_spacegroup(supercell, symprec=[1e-1, 1e-2, 1e-3], angle_tolerance=5.0)

    y_pred_sc1.append(supercell.info['spacegroup_nb_0.1'])
    y_pred_sc2.append(supercell.info['spacegroup_nb_0.01'])
    y_pred_sc3.append(supercell.info['spacegroup_nb_0.001'])


INFO: Using optimal supercell algorithm for replica determination. 
INFO: Noise realization: min: -0.0177999332115; max: 0.0178478044988
INFO: Using optimal supercell algorithm for replica determination. 
INFO: Noise realization: min: -0.02992767466; max: 0.0298029415834
INFO: Using optimal supercell algorithm for replica determination. 
INFO: Noise realization: min: -0.0179928063008; max: 0.0179137119533
INFO: Using optimal supercell algorithm for replica determination. 
INFO: Noise realization: min: -0.0179245330437; max: 0.0180366174408
INFO: Using optimal supercell algorithm for replica determination. 
INFO: Noise realization: min: -0.0299909186255; max: 0.0299050360257


In [59]:
for structure in ase_atoms_list_supercell:
    print(structure.info.keys())

['spacegroup_nb_0.01', u'spacegroup_1e-06', u'spacegroup_1e-09', u'label', 'spacegroup_nb_0.1', u'spacegroup_0.001', 'spacegroup_nb_0.001']
['spacegroup_nb_0.01', u'spacegroup_1e-06', u'spacegroup_1e-09', u'label', 'spacegroup_nb_0.1', u'spacegroup_0.001', 'spacegroup_nb_0.001']
['spacegroup_nb_0.01', u'spacegroup_1e-06', u'spacegroup_1e-09', u'label', 'spacegroup_nb_0.1', u'spacegroup_0.001', 'spacegroup_nb_0.001']
['spacegroup_nb_0.01', u'spacegroup_1e-06', u'spacegroup_1e-09', u'label', 'spacegroup_nb_0.1', u'spacegroup_0.001', 'spacegroup_nb_0.001']
['spacegroup_nb_0.01', u'spacegroup_1e-06', u'spacegroup_1e-09', u'label', 'spacegroup_nb_0.1', u'spacegroup_0.001', 'spacegroup_nb_0.001']


In [60]:
y_true = [221]*len(ase_atoms_list)
#accuracy_score(y_pred, y_pred_sc)
accuracy_score(y_true, y_pred_sc1), accuracy_score(y_true, y_pred_sc2), accuracy_score(y_true, y_pred_sc3)

(0.4, 0.0, 0.0)

In [61]:
Counter(y_true), Counter(y_pred_sc1), Counter(y_pred_sc2), Counter(y_pred_sc3)

(Counter({221: 5}),
 Counter({1: 2, 2: 1, 221: 2}),
 Counter({1: 5}),
 Counter({1: 5}))

In [63]:
from ase.visualize import view
idx=1
# change idx if you want to visualize another structure 
# idx=0 visualizes the first structure in the list
# Note: 0<=idx<len(ase_atoms_list)
view(ase_atoms_list_supercell[idx]*(1, 1, 1), viewer='x3d')
#view(ase_atoms_list[idx]*(3, 3, 3), viewer='x3d')